In [4]:
pip install unidecode scikit-learn BeautifulSoup4 pandas requests lxml

Note: you may need to restart the kernel to use updated packages.


In [5]:
from sklearn.preprocessing import StandardScaler
from requests.auth import HTTPBasicAuth
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from io import StringIO
import pandas as pd
import numpy as np
import unidecode
import requests
import locale
import pickle
import re 
import lxml

In [6]:
url_sc = 'https://www.senscritique.com/films/tops/top111'
page_sc = requests.get(url_sc)
soup = BeautifulSoup(page_sc.content, "html.parser")

In [11]:
def find_text(element, search_criteria):
    found_element = element.find(**search_criteria)
    return found_element.text if found_element else None

# Define a dictionary to map the keys to the search criteria
search_criteria_dict = {
    'original_title': {'name': 'p', 'attrs': {"class": "ProductListItem__OriginalTitle-sc-1jkxxpj-4 gATBvH"}},
    'rating': {'name': 'div', 'attrs': {"data-testid": "Rating"}},
    'director': {'name': 'a', 'attrs': {"data-testid": "link"}},
    'duration': {'name': 'span', 'attrs': {"data-testid": "duration"}},
    'genres': {'name': 'span', 'attrs': {"data-testid": "genres"}},
    'ranking': {'name': 'span', 'attrs': {"class": "ProductTitle__PositionWrapper-sc-mtf7b1-0 kcGHBE"}}
}

all_films_dict = {}
films = soup.find_all('div', class_="ProductListItem__Wrapper-sc-1jkxxpj-1 kusRkg")

for film in films:
    title = find_text(film, {'name': 'a', 'attrs': {"data-testid": "product-title"}})
    new_film_dict = {key: find_text(film, criteria) for key, criteria in search_criteria_dict.items()}

    link = film.find('a', {"data-testid": "product-title"})['href']
    
    all_films_dict[title] = new_film_dict

print(all_films_dict)


{'Douze Hommes en colère (1957)': {'original_title': None, 'rating': '8.7', 'director': 'Sidney Lumet', 'duration': '1 h 36 min. ', 'genres': 'Policier, Drame', 'ranking': None}, 'Harakiri (1962)': {'original_title': None, 'rating': '8.6', 'director': 'Masaki Kobayashi', 'duration': '2 h 13 min. ', 'genres': 'Drame', 'ranking': None}, 'Blade Runner : The Final Cut (2007)': {'original_title': None, 'rating': '8.6', 'director': 'Ridley Scott', 'duration': '1 h 57 min. ', 'genres': 'Science-fiction', 'ranking': None}, 'Le Bon, la Brute et le Truand (1966)': {'original_title': None, 'rating': '8.5', 'director': 'Sergio Leone', 'duration': '2 h 59 min. ', 'genres': 'Western, Aventure', 'ranking': None}, 'Barberousse (1965)': {'original_title': None, 'rating': '8.5', 'director': 'Akira Kurosawa', 'duration': '3 h 05 min. ', 'genres': 'Drame', 'ranking': None}, 'Les Sept Samouraïs (1954)': {'original_title': None, 'rating': '8.5', 'director': 'Akira Kurosawa', 'duration': '3 h 27 min. ', 'gen

In [32]:
film_single = soup.find('div', class_="ProductListItem__Wrapper-sc-1jkxxpj-1 kusRkg")

# Corrected the syntax for the find method
link = film_single.find('a', {"data-testid": "product-title"})['href']


def film_info(link):

    base_url = 'https://www.senscritique.com'
    film_url = base_url + link

    film_page = requests.get(film_url)
    film_page_soup = BeautifulSoup(film_page.content, "html.parser")

    extrainfo_dict = {}

    # Use find_all to get all matching elements
    film_year = 
    stats_elements = film_page_soup.select('p.Text__SCText-sc-1aoldkr-0.Stats__Text-sc-1u6v943-2.gATBvI')
    
    # Iterate through the list of elements and print the text content of each
    for element in stats_elements:
        print(element.text)

    number_of_critiques =   
    a_tag = soup.find('a', href='/film/douze_hommes_en_colere/370894/critiques')

    # Extract the text from the anchor tag
    text = a_tag.text  # "Critiques : avis d'internautes (592)"

    # Use string splitting and stripping to extract the number
    votes_text = text.split('(')[-1].split(')')[0]  # "592"
    votes = int(votes_text)  # 592

film_info(link)


51.7K
16.3K
6.8K
